In [72]:
from tqdm.notebook import trange, tqdm
import pickle as pkl
import numpy as np
from sage.symbolic.expression_conversions import PolynomialConverter

In [73]:
def mycollect(e1, e2):
    """
    Collects e2 coefficients in e1, even when e2 is not a factor of
    some of the terms of e1
    """
    g = maxima_calculus.gensym()._sage_()
    while g in e1.variables(): g = maxima_calculus.gensym()._sage_()
    return sum([v[0]*g^v[1]
                for v in e1.subs(e2==g).coefficients(g)]).subs(g==e2)

In [74]:
var_list=["a"+str(i) for i in range(0,100)]+["x"+str(i) for i in range(0,100)]+["s1","s2"]+["e"+str(i) for i in range(0,1000)]

In [75]:
var(var_list+["n0","s","t","n","q","x","a","b","c","d","u"])

(a0,
 a1,
 a2,
 a3,
 a4,
 a5,
 a6,
 a7,
 a8,
 a9,
 a10,
 a11,
 a12,
 a13,
 a14,
 a15,
 a16,
 a17,
 a18,
 a19,
 a20,
 a21,
 a22,
 a23,
 a24,
 a25,
 a26,
 a27,
 a28,
 a29,
 a30,
 a31,
 a32,
 a33,
 a34,
 a35,
 a36,
 a37,
 a38,
 a39,
 a40,
 a41,
 a42,
 a43,
 a44,
 a45,
 a46,
 a47,
 a48,
 a49,
 a50,
 a51,
 a52,
 a53,
 a54,
 a55,
 a56,
 a57,
 a58,
 a59,
 a60,
 a61,
 a62,
 a63,
 a64,
 a65,
 a66,
 a67,
 a68,
 a69,
 a70,
 a71,
 a72,
 a73,
 a74,
 a75,
 a76,
 a77,
 a78,
 a79,
 a80,
 a81,
 a82,
 a83,
 a84,
 a85,
 a86,
 a87,
 a88,
 a89,
 a90,
 a91,
 a92,
 a93,
 a94,
 a95,
 a96,
 a97,
 a98,
 a99,
 x0,
 x1,
 x2,
 x3,
 x4,
 x5,
 x6,
 x7,
 x8,
 x9,
 x10,
 x11,
 x12,
 x13,
 x14,
 x15,
 x16,
 x17,
 x18,
 x19,
 x20,
 x21,
 x22,
 x23,
 x24,
 x25,
 x26,
 x27,
 x28,
 x29,
 x30,
 x31,
 x32,
 x33,
 x34,
 x35,
 x36,
 x37,
 x38,
 x39,
 x40,
 x41,
 x42,
 x43,
 x44,
 x45,
 x46,
 x47,
 x48,
 x49,
 x50,
 x51,
 x52,
 x53,
 x54,
 x55,
 x56,
 x57,
 x58,
 x59,
 x60,
 x61,
 x62,
 x63,
 x64,
 x65,
 x66,
 x67,
 x68,
 x69,


In [76]:
e = SymmetricFunctions(QQ).e()

In [77]:
def ele_dict(deg):
        es=[e[i].expand(deg)*(-1)**i for i in range(1,deg+1)]
        coe=list(reversed([eval("a"+str(i)) for i in range(0,deg)]))
        return dict(zip(coe,es))

In [78]:
def poly(n):
    return sum([x0^(j)*eval("a"+str(j)) for j in [0..n-1]])+x0^n

In [79]:
def ordi(exp,vari):
    terms=exp.operands()
    lonely_terms=[i for i in terms if not(i.has(vari))]
    fixed=[i for i in terms if i not in lonely_terms]+[sum(lonely_terms)]
    return fixed

In [80]:
def red(op,deg):
    po=poly(deg).operands()
    po[0:2]=list(reversed(po[0:2]))
    if deg>=4:
        for i in range(0,deg-3):
            leading=op[i].subs({x0:1})
            new=[leading*j*(x0^(deg-4-i)) for j in po]
            if -(deg-4-i)<0:
                op=[(x-y).full_simplify().factor() for x,y in zip(op[0:-(deg-4-i)],new)]+op[len(op)-(deg-4-i):len(op)]
            else:
                op=[(x-y).full_simplify().factor() for x,y in zip(op,new)]
            op=op[1:]
        return op
    else:
        return op
        

In [81]:
def red2(op,deg):
    po=poly(deg)
    if deg>=4:
        for i in range(0,deg-3):
            leading=op.operands()[0].subs({x0:1})
            op=op-leading*(x0^(deg-4-i))*po
            op=mycollect(op.full_simplify(),x0)
        return op
    else:
        return op

In [82]:
def asum(n,roo):
    return sum([(n-j-1)*(n-j)/2*eval("x"+str(roo))^(n-j-2)*eval("a"+str(n-j)) for j in [1..n-2]])+n*(n-1)/2*eval("x"+str(roo))^(n-2)
def bsum(n,roo):
    return sum([j*(n-j)*eval("x"+str(roo))^(n-j-1)*eval("a"+str(n-j)) for j in [1..n-1]])
def csum(n,roo):
    return sum([(j-1)*(j)/2*eval("x"+str(roo))^(n-j)*eval("a"+str(n-j)) for j in [2..n]])

In [83]:
for j in range(2,7):
    formula=bsum(j,0)**2-4*asum(j,0)*csum(j,0)
    formula2=(mycollect(formula.subs({x0:x}),x)/(j-1)^2).full_simplify()
    che=formula.subs(ele_dict(j)).full_simplify().factor()
    #p1 = PolynomialConverter(formula2, base_ring=R)
    #print(p1())
    print(mycollect(formula/(j-1)^2,x0))
    print(che)

a1^2 - 4*a0
(x0 - x1)^2
(a2^2 - 3*a1)*x0^2 + a1^2 - 3*a0*a2 + (a1*a2 - 9*a0)*x0
4*(x0 - x1)^2*(x0 - x2)^2
1/3*(3*a3^2 - 8*a2)*x0^4 + 4/3*(a2*a3 - 6*a1)*x0^3 + 2/3*(2*a2^2 - 3*a1*a3 - 24*a0)*x0^2 + a1^2 - 8/3*a0*a2 + 4/3*(a1*a2 - 6*a0*a3)*x0
9*(x0 - x1)^2*(x0 - x2)^2*(x0 - x3)^2
1/2*(2*a4^2 - 5*a3)*x0^6 + 3/2*(a3*a4 - 5*a2)*x0^5 + 3/2*(a3^2 - a2*a4 - 10*a1)*x0^4 + (2*a2*a3 - 7*a1*a4 - 25*a0)*x0^3 + 3/2*(a2^2 - a1*a3 - 10*a0*a4)*x0^2 + a1^2 - 5/2*a0*a2 + 3/2*(a1*a2 - 5*a0*a3)*x0
16*(x0 - x1)^2*(x0 - x2)^2*(x0 - x3)^2*(x0 - x4)^2
1/5*(5*a5^2 - 12*a4)*x0^8 + 4/5*(2*a4*a5 - 9*a3)*x0^7 + 2/5*(4*a4^2 - 3*a3*a5 - 36*a2)*x0^6 + 4/5*(3*a3*a4 - 8*a2*a5 - 30*a1)*x0^5 + 1/5*(9*a3^2 - 4*a2*a4 - 70*a1*a5 - 180*a0)*x0^4 + 4/5*(3*a2*a3 - 8*a1*a4 - 30*a0*a5)*x0^3 + 2/5*(4*a2^2 - 3*a1*a3 - 36*a0*a4)*x0^2 + a1^2 - 12/5*a0*a2 + 4/5*(2*a1*a2 - 9*a0*a3)*x0
25*(x0 - x1)^2*(x0 - x2)^2*(x0 - x3)^2*(x0 - x4)^2*(x0 - x5)^2


In [84]:
f=(a2^2 - 3*a1)
g=(a1*a2 - 9*a0)
h=(a1^2 - 3*a0*a2)

In [85]:
#f=f.subs({a0:-1})
#g=g.subs({a0:-1})
#h=h.subs({a0:-1})

In [86]:
fsub=f.subs(ele_dict(3))
print(fsub)
gsub=g.subs(ele_dict(3))
print(gsub)
hsub=h.subs(ele_dict(3))
print(hsub)

(x0 + x1 + x2)^2 - 3*x0*x1 - 3*x0*x2 - 3*x1*x2
9*x0*x1*x2 - (x0*x1 + x0*x2 + x1*x2)*(x0 + x1 + x2)
-3*(x0 + x1 + x2)*x0*x1*x2 + (x0*x1 + x0*x2 + x1*x2)^2


In [87]:
action=(a*t+b)/(c*t+d)

In [88]:
action_dict={x0:t,x1:action, x2: action.subs({t:action}).full_simplify()}

In [89]:
action_dict

{x0: t,
 x1: (a*t + b)/(c*t + d),
 x2: (a*b + b*d + (a^2 + b*c)*t)/(b*c + d^2 + (a*c + c*d)*t)}

In [90]:
fact=fsub.subs(action_dict).full_simplify()
gact=gsub.subs(action_dict).full_simplify()
hact=hsub.subs(action_dict).full_simplify()

In [91]:
trans1=(gact/fact).full_simplify().factor()

In [92]:
trans2=(hact/fact).full_simplify().factor()

In [93]:
mycollect(trans2.denominator(),t)

b^2*c^2 - a*b*c*d + a^2*d^2 + b*c*d^2 + a*d^3 + d^4 + (a^2*c^2 + 2*a*c^2*d + c^2*d^2)*t^2 + (a*b*c^2 + a^2*c*d + b*c^2*d + 3*a*c*d^2 + 2*c*d^3)*t

In [94]:
eqx=(a^2*c^2 + 2*a*c^2*d + c^2*d^2)==1
eqy=(a*b*c^2 + a^2*c*d + b*c^2*d + 3*a*c*d^2 + 2*c*d^3)==0
eqz=(b^2*c^2 - a*b*c*d + a^2*d^2 + b*c*d^2 + a*d^3 + d^4 ==3*d^2)

In [95]:
solvi=solve([eqx,eqy,eqz],a,b,c)

In [96]:
solvi

[[a == -d - 1, b == -d^2 + d, c == 1], [a == -d + 1, b == d^2 + d, c == -1], [a == -d - 1, b == d^2 - d, c == -1], [a == -d + 1, b == -d^2 - d, c == 1]]

In [97]:
action.subs(solvi[0]).full_simplify()#.subs({d:i*t}).full_simplify()

-(d^2 + (d + 1)*t - d)/(d + t)

In [118]:
trans2.denominator().subs(solvi[0]).full_simplify()#.subs({d:1})

3*d^2 + t^2

In [119]:
trans1.numerator().subs(solvi[0]).full_simplify().factor()#.subs({d:1})

-2*(2*d + 1)*(d - t)*t

In [120]:
trans2.numerator().subs(solvi[0]).full_simplify().factor()#.subs({d:1})

d^4 + 4*d^3*t + 7*d^2*t^2 - 2*d^3 - 2*d^2*t + 4*d*t^2 + d^2 - 2*d*t + t^2

In [116]:
solve([eq1,eq2],d,t)

[[d == (-0.2835553179723988 + 0.3862317151651768*I), t == (-3.349020522808981 - 0.09810455569811992*I)], [d == (-0.2835553179723988 - 0.3862317151651768*I), t == (-3.34902052280898 + 0.09810455569811993*I)], [d == (-0.2471322818066755 + 0.5061746872473919*I), t == (3.40551884744559 + 0.1101965172320723*I)], [d == (-0.2471322818066755 - 0.5061746872473919*I), t == (3.40551884744559 - 0.1101965172320722*I)], [d == (-1.979453563086264 + 0.1646645977635815*I), t == (0.8351611006948684 + 1.170837307267175*I)], [d == (-1.979453563086264 - 0.1646645977635815*I), t == (0.8351611006948682 - 1.170837307267175*I)], [d == (1.938712591436766 + 0.02479194371249815*I), t == (-0.2488022824743343 + 0.579549990019579*I)], [d == (1.938712591436766 - 0.02479194371249815*I), t == (-0.2488022824743343 - 0.5795499900195789*I)]]